In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
from src.dataset.vtt import VTTDataset
import torch

In [ ]:
dataset = VTTDataset(
    meta_path="meta/vtt_integrated.jsonl",
    load_trans_frames=True,
    transform_cfg=dict(resize=True, random_crop=True, random_hflip=True),
    prefix_start=False
)

In [ ]:
x = torch.arange(24).reshape(2,3,4)

In [ ]:
z = torch.tensor([0,1,2,3,0,1])

In [ ]:
x[range(2), range(3), z]

In [ ]:
res = dataset[1]

In [ ]:
res["states_mask"]

In [ ]:
res["label_ids"][range(12), torch.sum(res["label_mask"], dim=-1)-1]

In [ ]:
res["label_ids"].shape

In [ ]:
res["label_ids"][0]

In [ ]:
res["label_mask"][0]

In [ ]:
dataset.tokenizer.decode(res["label_ids"][0].tolist())

In [ ]:
dataset.transform.to_pil(res["states"][0])

In [ ]:
res["states_mask"]

In [ ]:
dataset.transform.to_pil(res["states"][0])

In [ ]:
dataset.transform.to_pil(res["trans"][0][0])

In [ ]:
res["trans_mask"]

## Tokenizers

In [ ]:
from src.dataset.text import SimpleTokenizer
tokenizer = SimpleTokenizer()

In [ ]:
res = tokenizer.encode("And everyone wants to help with the new addition to the house.")
res

In [ ]:
tokenizer.decode(res)

In [ ]:
tokenizer.encode("<|endoftext|>")

In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [ ]:
tokenizer

In [ ]:
tokenizer.encode("<-start_of_text->")

In [ ]:
tokenizer.encode("And everyone wants to help with the new addition to the house.")

In [ ]:
from src.dataset.vtt import VTTDataset

In [ ]:
vtt = VTTDataset(return_raw_text=True)

In [ ]:
x = vtt[0]

In [ ]:
x["text"]

In [ ]:
x["label_ids"].shape

In [ ]:
from src.model.components.text_encoder import TextCLIP

In [ ]:
text_clip = TextCLIP("ViT-L/14")

In [ ]:
text_clip(x["label_ids"])

In [ ]:
x["label_mask"].sum(dim=-1) > 0

In [ ]:
from src.model.components.clip import clip

In [ ]:
model, _ = clip.load("ViT-L/14", device="cpu", download_root="/data/pretrain/clip")
text = clip.tokenize(x["text"])

In [ ]:
text.shape

In [ ]:
res_text_1 = text_clip(modified_text)
res_text_1

In [ ]:
res_text = model.encode_text(text)

In [ ]:
res_text

In [ ]:
torch.mean(res_text_1 * res_text, dim=-1)

In [ ]:
modified_text = text + (text == 0) * 49407
modified_text

In [ ]:
res_text

In [ ]:
res_text.shape

## Compute MAX WORDS

In [ ]:
from src.dataset.text import SimpleTokenizer
tokenizer = SimpleTokenizer()

In [ ]:
import src.utils.datatool as dtool

In [ ]:
data = dtool.read_jsonlines("/data/vtt/meta/vtt_integrated.jsonl")

In [ ]:
from tqdm.notebook import tqdm
n_words = []
for sample in tqdm(data):
    n_words.extend([len(tokenizer.encode(x['label'])) for x in sample['annotation']])

In [ ]:
max(n_words)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
axis = plt.hist(n_words, bins=21)